Fully Connected Neural Network

Crossvalidation

In [ ]:
import numpy as np
import sklearn
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
import datetime
from datetime import datetime

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import scipy
from scipy.stats import reciprocal
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import regularizers
from tensorflow.keras.layers import (Input, Convolution1D, Dense, MaxPooling1D,MaxPooling2D,AveragePooling2D,
                                    Flatten, Dropout, Activation, average,
                                    BatchNormalization, Reshape, Conv2D)
from tensorflow.keras.regularizers import l2
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import StandardScaler, MinMaxScaler, MaxAbsScaler, QuantileTransformer

from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.metrics import mean_absolute_error, make_scorer
from keras import backend as K
#scoring=make_scorer(mean_absolute_error, greater_is_better=False)

X           = np.load("rep.npy", allow_pickle=True)
energy      = np.load("energy.npy", allow_pickle=True)
X_training  = X[:1000]
Y_training  = energy[:1000]
X_test      = X[-1000:]
Y_test      = energy[-1000:]




def create_model(optimizer, dense_nparams, lr, decay, LAMBDA): #, ):
    #filters = 50
    if K.backend() == 'tensorflow':
        K.clear_session()

    K.clear_session()

    init = 'he_normal'
    
    
    regu = tf.keras.regularizers.l2(LAMBDA)


    model = Sequential()
    model.add(Dense(276, input_dim=276, kernel_initializer=init,kernel_regularizer=regu ,activation='linear'))
    #276 = 12 x 23 
    BatchNormalization()
    
    #BatchNormalization()
    #model.add(exponential_layer)
    model.add(Dense(dense_nparams, kernel_initializer=init, activation='selu'))
    BatchNormalization()
    model.add(Dense(dense_nparams, kernel_initializer=init, activation='selu'))
    BatchNormalization()
    model.add(Dense(dense_nparams, kernel_initializer=init, activation='selu'))
    BatchNormalization()

    model.add(Dense(dense_nparams, kernel_initializer=init, activation='selu'))
    BatchNormalization()
    #Dropout(rate=0.8)
    model.add(Dense(dense_nparams, kernel_initializer=init, activation='selu'))
    BatchNormalization()
    #Dropout(rate=0.5)
    model.add(Dense(dense_nparams, kernel_initializer=init, activation='selu'))
    BatchNormalization()
    #Dropout(rate=0.4)
    model.add(Dense(dense_nparams, kernel_initializer=init, activation='selu'))
    BatchNormalization()
    model.add(Dense(dense_nparams, kernel_initializer=init, activation='selu'))
    BatchNormalization()
    model.add(Dense(dense_nparams, kernel_initializer=init, activation='selu'))
    BatchNormalization()
    #Dropout(rate=0.1)
    model.add(Dense(dense_nparams, kernel_initializer=init, activation='selu'))
    BatchNormalization()
    #Dropout(rate=0.1)
    model.add(Dense(dense_nparams, kernel_initializer=init, activation='linear'))
    model.add(Dense(1, activation='linear', name='Output_Energy'))

    opt = Adam(lr=lr ,decay=decay)
    model.compile(loss='mae', optimizer=opt)
    
    #model.summary()
    return model


#kears_estimator = KerasRegressor(build_fn=create_model , epochs=1500, batch_size=25, verbose=2)
kears_estimator = KerasRegressor(build_fn=create_model , epochs=2500,  batch_size=25, verbose=2)



#param_grid = {
#    'dense_nparams': [400, 500, 600, 700],
#    'optimizer':['Adam'], 'lr': reciprocal(1e-6, 1e-2), 'decay': reciprocal(1e-5,1e-2), 'LAMBDA':  [1e-8, 1e-7], 'filters':  [2, 5, 10, 50 ], 'kernel_size':[3, 5, 10, 20, 50] }


#param_grid = {
#    'dense_nparams': [400, 500, 600, 700],
#    'optimizer':['Adam'], 'lr': [1e-6, 1e-2], 'decay': [1e-5,1e-2], 'LAMBDA':  [1e-8, 1e-7], 'filters':  [2, 5, 10, 50 ], 'kernel_size':[3, 5, 10, 20, 50] }


param_grid = {
    'dense_nparams': [3000, 3250],
    'optimizer':['Adam'], 'lr': [0.00055, 0.0006,0.00065,0.00075 ], 'decay': [0.008, 0.005, 0.004, 0.003, 0.0025,0.002,0.001, 0.0007, 0.0005, 0.00025], 'LAMBDA':  [1e-9,1e-8, 1e-7, 1e-6] }
#out_57:Best: -14.518220 using {'optimizer': 'Adam', 'lr': 0.0006, 'dense_nparams': 3000, 'decay': 0.0025, 'LAMBDA': 1e-0

kfold_splits = 5

#RandomizedSearchCV

#search = GridSearchCV(estimator=kears_estimator,  param_grid=param_grid,cv=kfold_splits, verbose=1)
#search  = RandomizedSearchCV(estimator=kears_estimator, param_distributions=param_grid, n_iter=10, cv=kfold_splits,scorer=scoring,  scoring='neg_mean_absolute_error', verbose=10)
#search  = RandomizedSearchCV(estimator=kears_estimator, param_distributions=param_grid, n_iter=10, cv=kfold_splits,scoring=scoring, verbose=10)
search  = RandomizedSearchCV(estimator=kears_estimator, param_distributions=param_grid, n_iter=10, cv=kfold_splits, verbose=10)





my_callbacks =  [keras.callbacks.EarlyStopping(patience=100), keras.callbacks.TerminateOnNaN() ]
#https://medium.com/@am.benatmane/keras-hyperparameter-tuning-using-sklearn-pipelines-grid-search-with-cross-validation-ccfc74b0ce9f

grid_result = search.fit(X_training, Y_training, callbacks= my_callbacks,validation_data=(X_test, Y_test) ) 

#,n_jobs=-1


# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
print ("rest")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

We find the following hyperparameters to perform well with this network

             -14.518220 using {'optimizer': 'Adam', 'lr': 0.0006, 'dense_nparams': 3000, 'decay': 0.0025, 'LAMBDA': 1e-0
out_29:Best: -14.927439 using {'optimizer': 'RMSprop', 'lr': 0.00075, 'dense_nparams': 3000, 'decay': 0.002, 'LAMBDA': 1e-06}
out_38:Best: -14.959991 using {'optimizer': 'Adam', 'lr': 0.0006, 'dense_nparams': 3000, 'decay': 0.002, 'LAMBDA': 1e-09}
out_40:Best: -14.744321 using {'optimizer': 'Adam', 'lr': 0.0006, 'dense_nparams': 3250, 'decay': 0.0007, 'LAMBDA': 1e-06}
out_50:Best: -14.992078 using {'optimizer': 'Adam', 'lr': 0.0006, 'dense_nparams': 3000, 'decay': 0.0025, 'LAMBDA': 1e-07}
out_53:Best: -14.988046 using {'optimizer': 'Adam', 'lr': 0.0006, 'dense_nparams': 3000, 'decay': 0.002, 'LAMBDA': 1e-06}
out_57:Best: -14.518220 using {'optimizer': 'Adam', 'lr': 0.0006, 'dense_nparams': 3000, 'decay': 0.0025, 'LAMBDA': 1e-06}
out_61:Best: -14.798697 using {'optimizer': 'Adam', 'lr': 0.00055, 'dense_nparams': 3000, 'decay': 0.002, 'LAMBDA': 1e-07}
out_62:Best: -14.446813 using {'optimizer': 'Adam', 'lr': 0.0006, 'dense_nparams': 3250, 'decay': 0.001, 'LAMBDA': 1e-07}
out_64:Best: -14.874572 using {'optimizer': 'Adam', 'lr': 0.00075, 'dense_nparams': 3000, 'decay': 0.002, 'LAMBDA': 1e-08}
out_66:Best: -14.908051 using {'optimizer': 'Adam', 'lr': 0.00075, 'dense_nparams': 3000, 'decay': 0.002, 'LAMBDA': 1e-09}
out_67:Best: -14.645539 using {'optimizer': 'Adam', 'lr': 0.00055, 'dense_nparams': 3000, 'decay': 0.0007, 'LAMBDA': 1e-09}
out_68:Best: -14.875695 using {'optimizer': 'Adam', 'lr': 0.0006, 'dense_nparams': 3000, 'decay': 0.002, 'LAMBDA': 1e-09}
out_69:Best: -14.528221 using {'optimizer': 'Adam', 'lr': 0.00055, 'dense_nparams': 3250, 'decay': 0.0007, 'LAMBDA': 1e-09}
out_73:Best: -14.848996 using {'optimizer': 'Adam', 'lr': 0.0006, 'dense_nparams': 3250, 'decay': 0.002, 'LAMBDA': 1e-07}
out_74:Best: -14.613671 using {'optimizer': 'Adam', 'lr': 0.00055, 'dense_nparams': 3250, 'decay': 0.001, 'LAMBDA': 1e-06}
out_75:Best: -14.840783 using {'optimizer': 'Adam', 'lr': 0.00055, 'dense_nparams': 3250, 'decay': 0.002, 'LAMBDA': 1e-06}
out_77:Best: -14.818040 using {'optimizer': 'Adam', 'lr': 0.00055, 'dense_nparams': 3000, 'decay': 0.0005, 'LAMBDA': 1e-06}
out_78:Best: -14.921816 using {'optimizer': 'Adam', 'lr': 0.0006, 'dense_nparams': 3250, 'decay': 0.003, 'LAMBDA': 1e-07}
out_80:Best: -14.774423 using {'optimizer': 'Adam', 'lr': 0.00075, 'dense_nparams': 3000, 'decay': 0.0025, 'LAMBDA': 1e-09}
out_81:Best: -14.997505 using {'optimizer': 'Adam', 'lr': 0.00055, 'dense_nparams': 3250, 'decay': 0.001, 'LAMBDA': 1e-06}
out_83:Best: -14.653083 using {'optimizer': 'Adam', 'lr': 0.0006, 'dense_nparams': 3000, 'decay': 0.0025, 'LAMBDA': 1e-06}
out_84:Best: -14.812062 using {'optimizer': 'Adam', 'lr': 0.00065, 'dense_nparams': 3250, 'decay': 0.001, 'LAMBDA': 1e-06}
out_85:Best: -14.838064 using {'optimizer': 'Adam', 'lr': 0.00065, 'dense_nparams': 3250, 'decay': 0.001, 'LAMBDA': 1e-06}
out_87:Best: -14.971521 using {'optimizer': 'Adam', 'lr': 0.00075, 'dense_nparams': 3000, 'decay': 0.003, 'LAMBDA': 1e-06}
out_91:Best: -14.599177 using {'optimizer': 'Adam', 'lr': 0.00075, 'dense_nparams': 3250, 'decay': 0.0025, 'LAMBDA': 1e-07}
out_92:Best: -14.734776 using {'optimizer': 'Adam', 'lr': 0.00055, 'dense_nparams': 3250, 'decay': 0.003, 'LAMBDA': 1e-07}
out_96:Best: -14.947271 using {'optimizer': 'Adam', 'lr': 0.00065, 'dense_nparams': 3000, 'decay': 0.003, 'LAMBDA': 1e-06}
